In [1]:
import numpy as np
import re

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot
from pol_analysis.getStokes import GetStokes

In [2]:
def run_all(obj, band, star_coords, stddev_0_vals=None, bhd_coords=(1021.5, 70.), show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, special_ob_pairs=None, only_seeing_fit=False, centroid_box_size=5, pdata_filenames_for_masks=None, append_output=False):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #BHD coords
    bhd_ex_ref, bhd_ey_ref = bhd_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)

    #Now, go through all the OB pairs.
    ob_combs = list()
    for ob_pair in bhd.ob_pairs:
        ob_combs.append([[ob_pair[0]],[ob_pair[1]]])
    if len(bhd.ob_pairs)>1:
        ob_combs.append([None, None])

    #Add special combinations if provided. 
    if special_ob_pairs is not None:
        ob_combs.extend(special_ob_pairs)

    if append_output: 
        cato = open("nearby_star_{}_pol_{}.dat".format(obj,  band), "a")
    else:
        cato = open("nearby_star_{}_pol_{}.dat".format(obj,  band), "w")
    for k, ob_comb in enumerate(ob_combs):
        ob_ids = ob_comb[0]
        mjds = ob_comb[1]
        #print(ob_ids, mjds)
        #continue

        #Get the seeing. 
        fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
        if stddev_0_vals is not None:
            stddev_0 = stddev_0_vals[k]
        else:
            stddev_0 = np.ones(len(fnames))*1.1
        phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=ob_ids, mjds=mjds)
        #print(phot.seeing)
        if only_seeing_fit:
            continue

        #Get the photometry convolving to a common PSF. 
        phot.get_phot(r_ap=1.0, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

        #Measure the stokes and polarization parameters. 
        stk_obj = GetStokes(bhd)
        stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

        #Get the polarization values for the BHD. 
        epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
        dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
        k = np.argmin(dist2)
        #print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

        str = "{:5.1f} {:5.1f}".format(epos_ref[k,0], epos_ref[k,1])
        str += " {:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k])
        str += " {:5.1f} {:5.1f} {:5.1f}".format(np.min(phot.seeing), np.mean(phot.seeing), np.max(phot.seeing))
        if ob_ids is None:
            str += " All"
        else:
            for i, ob_id in enumerate(ob_ids):
                str += " {:10d} {:10d}".format(ob_id, mjds[i])
        print(str)
        cato.write(str+"\n")
    cato.close()

In [3]:
#Read the coordinates of the reference stars.
star_coords = dict()
cat = open("reference_stars.txt")
for line in cat:
    x = line.split()
    star_coords[x[0]] = (float(x[1]), float(x[2]))
cat.close()

## W0019-1046

In [4]:
# wid = "W0019-1046"
# run_all(wid, "R_SPECIAL", star_coords=star_coords[wid])

## W0204-0506

In [ ]:
wid = "W0204-0506"
#This is a bright star in the field.
run_all(wid, "R_SPECIAL", star_coords=star_coords[wid], bhd_coords=(1086., 223.))

mkdir: work: File exists


1086.1 222.8  0.07  0.07   87.2   64.0   0.7   0.8   0.9    3565504      60209
1086.1 222.9  0.15  0.08  127.7   16.6   0.5   0.6   0.7    3565580      60207
1086.1 222.8  0.11  0.06  113.1   50.8   0.5   0.7   0.9 All


In [ ]:
#This is a nearby star. 
run_all(wid, "R_SPECIAL", star_coords=star_coords[wid], bhd_coords=(1013., 55.), append_output=True)

mkdir: work: File exists


1011.9  53.8  1.26  0.79  139.6   21.7   0.7   0.8   0.9    3565504      60209
1011.8  53.7  0.89  0.67   67.3   52.0   0.5   0.6   0.7    3565580      60207
1011.9  53.8  0.38  0.46  120.4   55.1   0.5   0.7   0.9 All


In [ ]:
#This is the BHD
run_all(wid, "R_SPECIAL", star_coords=star_coords[wid], append_output=True)

mkdir: work: File exists


1020.2  69.8 25.20  1.00   12.4    1.2   0.7   0.8   0.9    3565504      60209
1020.2  69.8 24.30  0.85   13.0    1.0   0.5   0.6   0.7    3565580      60207
1020.2  69.8 24.72  0.67   12.7    0.7   0.5   0.7   0.9 All


## W0220+0137

In [7]:
# wid = "W0220+0137"
# run_all(wid, "R_SPECIAL", star_coords=star_coords[wid])

## W0831+0140

In [8]:
# wid = "W0831+0140"
# run_all(wid, "R_SPECIAL", star_coords=star_coords[wid])

## W0116-0505

#### $R_{\rm SPECIAL}$

In [9]:
# wid = "W0116-0505"
# run_all(wid, "R_SPECIAL", star_coords=star_coords[wid])#, only_seeing_fit=True, show_seeing_plots=True)#, centroid_box_size=5, phot_force_new=True)#, pdata_force_new=True)

#### $I_{\rm BESS}$

In [10]:
# special_ob_pairs = list()
# special_ob_pairs.append([[3564862, 3565005],[60201, 60148]])
# special_ob_pairs.append([[3564862, 3565005, 3564862], [60201, 60148, 60148]])

# run_all(wid, "I_BESS", star_coords=star_coords[wid], special_ob_pairs=special_ob_pairs)#, only_seeing_fit=True, show_seeing_plots=True)#

#### $v_{\rm HIGH}$

In [11]:
# #The images are not good for creating the masks. Instead, we will use the i-band images obtained closest in time to generate the masks. 
# aux = PolData("W0116-0505",bband="I_BESS")
# filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60146])

# run_all(wid, "v_HIGH", star_coords=star_coords[wid], pdata_filenames_for_masks=filenames_for_masks)#, only_seeing_fit=True, show_seeing_plots=True, phot_force_new=True, pdata_force_new=True)#